In [ ]:
import numpy as np
import os
import jsonlines
import numpy as np
import torch

In [ ]:
# What need to be collect

# for each single event: 
# 1. source nodes
# 2. destination nodes
# 3. time_stamps
# 4. raw_node_features
# 5. edge_idxs
# *: Neighbor_finder, need time_span. 

'''
Time span should be the interval from the source post to current. Therefore,
it's equivalent to my collected time_stamps. 

And you will need a pretrained BERT to collect raw_node_features
'''

In [ ]:
# Load training data as train_data， write into train_file, test_file and label_file
# Load PHEME dataset. 


path = 'project-data/'
train_data_path = 'train.data.jsonl'
dev_data_path = 'dev.data.jsonl'
test_data_path = 'test.data.jsonl'

train_data = []
with jsonlines.open(path + train_data_path) as reader:
    for obj in reader:
        train_data.append(obj)
print('length of traininig data:', len(train_data))

        
# load the development data as dev_data(used as test_data in RvNN)
dev_data = []
with jsonlines.open(path + dev_data_path) as reader1:
    for obj in reader1:
        dev_data.append(obj)
print('length of devolop data: ', len(dev_data))


test_data = []
with jsonlines.open(path + test_data_path) as reader2:
    for obj in reader2:
        test_data.append(obj)
print('length of test data: ', len(test_data))

raw_PHEME = train_data + dev_data + test_data
print('length of whole data: ', len(raw_PHEME))

import json

label_path = 'PHEME_label.json'
with open(path+label_path) as f:
    PHEME_label = json.load(f)




length of traininig data: 4641
length of devolop data:  580
length of test data:  581
length of whole data:  5802


In [ ]:
def time_stamp(dic):
    # given a tweet dict
    month_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', "Aug", 'Sep', 'Oct', 'Nov', 'Dec']
    time = dic['created_at']
    month = time[4:7]
    day = time[8:10]
    h = time[11:13]
    m = time[14:16]
    s = time[17:19]
    return month_list.index(month)*86400*30 + int(day)*86400 + int(h)*3600 + int(m)*60 + int(s)*1

'''
for idx in range(len(train_data)):
    train_data[idx] = sorted(train_data[idx], key=lambda tree_node: time_stamp(tree_node))

for idx in range(len(test_data)):
    test_data[idx] = sorted(test_data[idx], key=lambda tree_node: time_stamp(tree_node))

for idx in range(len(dev_data)):
    dev_data[idx] = sorted(dev_data[idx], key=lambda tree_node: time_stamp(tree_node))
'''
for idx in range(len(raw_PHEME)):
    raw_PHEME[idx] = sorted(raw_PHEME[idx], key=lambda tree_node: time_stamp(tree_node))

In [ ]:
dataset = {}
for item in raw_PHEME:
    ID = item[0]['id_str']
    dataset[ID] = {}
    
    

In [ ]:
print(raw_PHEME[0][0].keys())
print(raw_PHEME[0][1].keys())

dict_keys(['contributors', 'truncated', 'text', 'in_reply_to_status_id', 'id', 'favorite_count', 'source', 'retweeted', 'coordinates', 'entities', 'in_reply_to_screen_name', 'id_str', 'retweet_count', 'in_reply_to_user_id', 'favorited', 'user', 'geo', 'in_reply_to_user_id_str', 'possibly_sensitive', 'lang', 'created_at', 'in_reply_to_status_id_str', 'place', 'extended_entities'])
dict_keys(['contributors', 'truncated', 'text', 'in_reply_to_status_id', 'id', 'favorite_count', 'source', 'retweeted', 'coordinates', 'entities', 'in_reply_to_screen_name', 'id_str', 'retweet_count', 'in_reply_to_user_id', 'favorited', 'user', 'geo', 'in_reply_to_user_id_str', 'lang', 'created_at', 'in_reply_to_status_id_str', 'place'])


In [ ]:
from transformers import BertModel
from transformers import BertTokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def bert_module(sentence):
    
    tokens = tokenizer.tokenize(sentence)
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    attn_mask = [1 for _ in range(len(tokens))]
    seg_ids = [0 for _ in range(len(tokens))]
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    token_ids_tensor = torch.tensor(token_ids).unsqueeze(dim=0)
    attn_mask_tensor = torch.tensor(attn_mask).unsqueeze(dim=0)
    seg_ids_tensor = torch.tensor(seg_ids).unsqueeze(dim=0)
    
    outputs = bert_model(token_ids_tensor, attention_mask=attn_mask_tensor,\
                         token_type_ids=seg_ids_tensor)
    
    #print(outputs.last_hidden_state.shape)
    
    return None

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
for event in raw_PHEME:
    for post in event:
        text = post['text']
        bert_module(text)

In [ ]:
def build_tree(event):
    # for return 
    sources = []
    destinations = []
    timestamps = []
    raw_node_features = np.zeros((len(event) + 1, 768))
    edge_indexs = np.array([i for i in range(1, len(event))])
    #  post ID to index:
    id2idx = {}
    idx = 1
    
    source_post = event[0]
    id2idx[source_post['id_str']] = idx
    raw_node_features[idx] = bert_module(source_post['text'])
    idx += 1    
    
    # id2idx should be down before data preprocessing, in case they are not chronological 
    for post in event[1:]:
        if post['id_str'] not in id2idx:
            id2idx[post['id_str']] = idx
            idx += 1
    # We should differentiate the source post and other reply posts. 
    for post in event[1:]:
        sources.append(id2idx[post['in_reply_to_status_id_str']])
        destinations.append(id2idx[post['id_str']])
        raw_node_features[id2idx[post['id_str']]] = bert_module(post['text'])
        
        timestamps.append(post['created_at'] - source_post['created_at'])
        
    return sources, destinations, timestamps, raw_node_features, edge_indexs
        